In [ ]:
!pip install pandas replicate

In [ ]:
import pandas as pd
import os
import json # Akan digunakan untuk memproses output JSON dari Granite
from google.colab import userdata
from replicate import Client
# Set the API token
api_token = userdata.get('REPLICATE_AI')
replicate_client = Client(api_token=api_token)

print("Lingkungan Google Colab siap!")

pd.read_csv('Womens Clothing E-Commerce Reviews.csv')


In [ ]:
import pandas as pd

df=pd.read_csv('Womens Clothing E-Commerce Reviews.csv')

print("Informasi Dataset Awal:")
print(df.info())
print("\n5 baris pertama:")
print(df.head())

In [ ]:
# Langkah-langkah pembersihan data dasar:
# - Hapus baris yang memiliki nilai kosong pada kolom 'Review Text'
df.dropna(subset=['Review Text'], inplace=True)

# - Hapus kolom yang tidak relevan untuk analisis sentimen, untuk menghemat memori
df.drop(columns=['Unnamed: 0', 'Title'], inplace=True)

# - Pastikan kolom 'Review Text' bertipe string
df['Review Text'] = df['Review Text'].astype(str)

print("\nInformasi Dataset Setelah Pembersihan:")
print(df.info())
print(f"\nJumlah ulasan yang akan dianalisis: {len(df)}")


In [ ]:
# Membuat fungsi untuk memanggil model Granite
import json # Import json here

def analyze_review_with_granite(review_text):
    """
    Fungsi ini mengambil teks ulasan dan mengirimkannya ke model IBM Granite
    untuk analisis sentimen, ekstraksi topik, dan ringkasan.
    Fungsi ini secara spesifik meminta output dalam format JSON.
    """

# Membuat prompt yang terstruktur dan rinci
    prompt_template = f"""
    Anda adalah seorang analis data yang bertugas menganalisis ulasan produk e-commerce.
    Lakukan analisis terhadap ulasan berikut dan berikan hasilnya dalam format JSON.

    Tugas Anda:
    1.  Klasifikasikan sentimen keseluruhan dari ulasan sebagai salah satu dari: "Positif", "Negatif", atau "Campuran".
    2.  Identifikasi dan ekstrak topik atau area fokus utama yang dibahas. Setiap topik harus diklasifikasikan sebagai "Positif" atau "Negatif" sesuai sentimennya. Contoh topik: "kualitas bahan", "ukuran", "harga", "pengiriman".
    3.  Tulis ringkasan singkat dari ulasan yang mencakup poin-poin terpenting.

    Ulasan Pelanggan:
    {review_text}

    Format Output JSON yang Diharapkan:
    {{
      "sentimen_keseluruhan": "...",
      "area_fokus": [
        {{"topik": "...", "sentimen": "..."}}
      ],
      "ringkasan": "..."
    }}
    """

    # Konfigurasi parameter model yang dioptimalkan
    # Kita menggunakan parameter yang lebih seimbang untuk hasil yang lebih baik
    parameters = {
        "max_tokens": 1024, # Nilai yang cukup besar untuk output lengkap
        "temperature": 0.5, # Membuat output lebih faktual dan stabil
        "top_p": 0.9,      # Mempertahankan variasi kata tanpa terlalu melenceng
        "repetition_penalty": 1.1, # Menghindari pengulangan kata
    }

    try:
        print(f"Menganalisis ulasan: {review_text[:50]}...")
        # Ensure replicate_client is defined globally or in a parent scope
        output_generator = replicate_client.run(
            "ibm-granite/granite-3.2-8b-instruct",
            input={"prompt": prompt_template, **parameters}
        )

        full_output = "".join(output_generator)

        # Penanganan kesalahan JSON yang lebih baik
        # Kadang model menghasilkan karakter tambahan (misalnya, `json)
        # Kita akan membersihkannya untuk memastikan dapat diurai.
        cleaned_output = full_output.strip().replace('```json', '').replace('```', '')
        json_output = json.loads(cleaned_output)
        return json_output

    except json.JSONDecodeError:
        print("Model gagal memberikan output JSON yang valid. Mengembalikan output mentah.")
        # Attempt to return raw output if JSON decoding fails
        return {"error": "Invalid JSON output", "raw_output": full_output if 'full_output' in locals() else "No output generated"}

    except Exception as e:
        print(f"Terjadi kesalahan saat memanggil model: {e}")
        return {"error": str(e)}

In [ ]:
# Ambil ulasan pertama dari data yang sudah kita bersihkan
# Pastikan Anda sudah menjalankan kode untuk memuat data sebelumnya
import json # Import json here
import pandas as pd # Make sure pandas is imported if not already

try:
    # Assuming df is already loaded and cleaned from previous cells
    # If not, uncomment the following lines:
    # df = pd.read_csv('Womens Clothing E-Commerce Reviews.csv')
    # df.dropna(subset=['Review Text'], inplace=True)

    # Ensure df is available from previous execution
    if 'df' not in locals() and 'df' not in globals():
        print("DataFrame 'df' not found. Please run the data loading and cleaning cells first.")
    else:
        sample_review = df['Review Text'].iloc[0]

        # Memanggil fungsi analisis kita
        # Make sure analyze_review_with_granite is defined in a previous cell and that cell has been run
        analysis_result = analyze_review_with_granite(sample_review)
        print("\n\nHasil Analisis Ulasan:")
        print(json.dumps(analysis_result, indent=2))

except FileNotFoundError:
    print("File data tidak ditemukan. Pastikan Anda sudah mengunggah 'Womens Clothing E-Commerce Reviews.csv' ke Colab.")
except IndexError:
    print("DataFrame kosong atau 'Review Text' column is missing after cleaning. Pastikan file CSV memiliki data yang valid dan pembersihan berjalan dengan benar.")
except NameError as ne:
    print(f"NameError: {ne}. Make sure analyze_review_with_granite function and replicate_client are defined and accessible.")
except Exception as e:
    print(f"An unexpected error occurred: {e}")

In [ ]:
# Mengambil sampel acak dari 100 ulasan dari DataFrame yang sudah dibersihkan
# Kita menggunakan random_state untuk memastikan sampel yang sama setiap kali kode dijalankan
sampled_reviews = df.sample(n=100, random_state=42)

print(f"Berhasil mengambil {len(sampled_reviews)} ulasan sebagai sampel untuk analisis.")

In [ ]:
import time

# List kosong untuk menyimpan hasil analisis dari AI
analysis_results = []

# Loop untuk memproses setiap ulasan dalam sampel
for index, row in sampled_reviews.iterrows():
    review_text = row['Review Text']

    # Memanggil fungsi analisis dan menambahkan hasilnya ke list
    result = analyze_review_with_granite(review_text)

    # Menyimpan ulasan asli dan hasil analisis AI
    result['original_review'] = review_text
    analysis_results.append(result)

    # Jeda singkat untuk menghindari batasan API (rate limiting)
    time.sleep(2)

print("\nProses analisis batch selesai!")

In [ ]:
import time

# List kosong untuk menyimpan hasil analisis dari AI
analysis_results = []

# Loop untuk memproses setiap ulasan dalam sampel
for index, row in sampled_reviews.iterrows():
    review_text = row['Review Text']

    # Tambahkan retry logic untuk mengatasi rate limiting
    max_retries = 5
    retry_count = 0
    success = False

    while not success and retry_count < max_retries:
        try:
            # Memanggil fungsi analisis dan menambahkan hasilnya ke list
            print(f"Menganalisis ulasan: {review_text[:50]}...")
            result = analyze_review_with_granite(review_text)

            # Jika berhasil, tandai sebagai sukses dan keluar dari loop
            result['original_review'] = review_text
            analysis_results.append(result)
            success = True

        except Exception as e:
            if "status: 429" in str(e):
                # Ekstrak waktu tunggu dari pesan error (contoh: "~7s")
                try:
                    wait_time = int(''.join(filter(str.isdigit, str(e))))
                    print(f"Rate limited. Menunggu {wait_time} detik dan mencoba lagi...")
                    time.sleep(wait_time + 1) # Tambahkan 1 detik untuk amannya
                except (ValueError, IndexError):
                    # Jika tidak bisa mengekstrak waktu, tunggu 10 detik
                    print("Rate limited. Tidak dapat mengekstrak waktu tunggu. Menunggu 10 detik...")
                    time.sleep(10)
                retry_count += 1
            else:
                print(f"Terjadi kesalahan lain: {e}. Mengabaikan ulasan ini.")
                analysis_results.append({"error": str(e), "original_review": review_text})
                success = True # Lanjut ke ulasan berikutnya meskipun ada error

# Setelah loop, buat DataFrame dari hasil
results_df = pd.DataFrame(analysis_results)

print("\nProses analisis batch selesai!")

# Menampilkan 5 baris pertama dari DataFrame baru
print("\nDataFrame Hasil Analisis AI:")
print(results_df.head())

# Menampilkan informasi tentang DataFrame baru
print(results_df.info())

In [ ]:
import time
import pandas as pd # Pastikan pandas diimpor jika belum
import json # Pastikan json diimpor jika belum

# ====================================================================
# Tahap 4: Analisis Ulasan Skala Besar dengan Perbaikan
# ====================================================================

# List kosong untuk menyimpan hasil analisis dari AI
analysis_results = []

# Mengambil sampel acak dari 100 ulasan dari DataFrame yang sudah dibersihkan
# Kita menggunakan random_state untuk memastikan sampel yang sama setiap kali kode dijalankan
sampled_reviews = df.sample(n=100, random_state=42)
print(f"Berhasil mengambil {len(sampled_reviews)} ulasan sebagai sampel untuk analisis.")

# Loop untuk memproses setiap ulasan dalam sampel
for index, row in sampled_reviews.iterrows():
    review_text = row['Review Text']

    # Menambahkan retry logic untuk mengatasi rate limiting
    max_retries = 5
    retry_count = 0
    success = False

    while not success and retry_count < max_retries:
        try:
            print(f"Menganalisis ulasan: {review_text[:50]}... (Percobaan ke-{retry_count + 1})")

            # Memanggil fungsi analisis kita
            result = analyze_review_with_granite(review_text)

            # Jika berhasil, tandai sebagai sukses dan keluar dari loop
            result['original_review'] = review_text
            analysis_results.append(result)
            success = True

        except Exception as e:
            if "status: 429" in str(e):
                try:
                    # Mencoba mengekstrak waktu tunggu dari pesan error
                    wait_time = int(''.join(filter(str.isdigit, str(e))))
                    # Tambahkan jeda waktu yang disarankan + sedikit ekstra
                    print(f"Rate limited. Menunggu {wait_time + 1} detik dan mencoba lagi...")
                    time.sleep(wait_time + 1)
                except (ValueError, IndexError):
                    # Jika gagal, jeda dengan aman selama 10 detik
                    print("Rate limited. Tidak dapat mengekstrak waktu tunggu. Menunggu 10 detik...")
                    time.sleep(10)
                retry_count += 1
            else:
                print(f"Terjadi kesalahan lain: {e}. Mengabaikan ulasan ini.")
                analysis_results.append({"error": str(e), "original_review": review_text})
                success = True

    # Jeda yang lebih panjang dan konsisten antar permintaan
    print("Menunggu 10 detik sebelum permintaan berikutnya...")
    time.sleep(10)


# ====================================================================
# Tahap 5: Pembuatan dan Analisis DataFrame Hasil
# ====================================================================

# Membuat DataFrame dari list hasil analisis
results_df = pd.DataFrame(analysis_results)

# Menampilkan 5 baris pertama dari DataFrame baru
print("\nProses analisis batch selesai!")
print("\nDataFrame Hasil Analisis AI:")
display(results_df.head())

# Menampilkan informasi tentang DataFrame baru
print("\nInformasi DataFrame Hasil:")
display(results_df.info())

In [ ]:
import time
import pandas as pd # Pastikan pandas diimpor jika belum
import json # Pastikan json diimpor jika belum

# ====================================================================
# Tahap 4: Analisis Ulasan Skala Besar dengan Perbaikan Akhir
# ====================================================================

# List kosong untuk menyimpan hasil analisis dari AI
analysis_results = []

# Mengambil sampel acak dari 100 ulasan dari DataFrame yang sudah dibersihkan
# Kita menggunakan random_state untuk memastikan sampel yang sama setiap kali kode dijalankan
sampled_reviews = df.sample(n=100, random_state=42)
print(f"Berhasil mengambil {len(sampled_reviews)} ulasan sebagai sampel untuk analisis.")

# Loop untuk memproses setiap ulasan dalam sampel
for index, review_text in sampled_reviews['Review Text'].items():

    # Menambahkan retry logic untuk mengatasi rate limiting dan error lainnya
    max_retries = 5
    retry_count = 0
    success = False

    while not success and retry_count < max_retries:
        try:
            print(f"Menganalisis ulasan: {review_text[:50]}... (Percobaan ke-{retry_count + 1})")

            # Memanggil fungsi analisis kita
            result = analyze_review_with_granite(review_text)

            # Jika berhasil, tambahkan ulasan asli dan tandai sukses
            result['original_review'] = review_text
            analysis_results.append(result)
            success = True

        except Exception as e:
            if "status: 429" in str(e):
                try:
                    # Mencoba mengekstrak waktu tunggu dari pesan error
                    wait_time = int(''.join(filter(str.isdigit, str(e))))
                    print(f"Rate limited. Menunggu {wait_time + 2} detik dan mencoba lagi...")
                    time.sleep(wait_time + 2) # Tambahkan 2 detik untuk amannya
                except (ValueError, IndexError):
                    print("Rate limited. Tidak dapat mengekstrak waktu tunggu. Menunggu 15 detik...")
                    time.sleep(15)
                retry_count += 1
            else:
                print(f"Terjadi kesalahan: {e}. Mencatat error dan melanjutkan...")
                analysis_results.append({"error": str(e), "original_review": review_text})
                success = True

    # Jeda yang lebih panjang dan konsisten antar permintaan untuk mencegah throttling
    if success:
        print("Analisis berhasil. Menunggu 10 detik sebelum permintaan berikutnya...")
        time.sleep(10)


# ====================================================================
# Tahap 5: Pembuatan dan Analisis DataFrame Hasil
# ====================================================================

# Membuat DataFrame dari list hasil analisis
results_df = pd.DataFrame(analysis_results)

# Menampilkan 5 baris pertama dari DataFrame baru
print("\nProses analisis batch selesai!")
print("\nDataFrame Hasil Analisis AI:")
display(results_df.head())

# Menampilkan informasi tentang DataFrame baru
print("\nInformasi DataFrame Hasil:")
display(results_df.info())

In [ ]:
# ====================================================================
# Tahap 4: Analisis Ulasan Skala Besar dengan Perbaikan Akhir
# ====================================================================

# List kosong untuk menyimpan hasil analisis dari AI
analysis_results = []

# Mengambil sampel acak dari 100 ulasan dari DataFrame yang sudah dibersihkan
# Kita menggunakan random_state untuk memastikan sampel yang sama setiap kali kode dijalankan
sampled_reviews = df.sample(n=100, random_state=42)
print(f"Berhasil mengambil {len(sampled_reviews)} ulasan sebagai sampel untuk analisis.")

# Loop untuk memproses setiap ulasan dalam sampel
for index, review_text in sampled_reviews['Review Text'].items():

    # Menambahkan retry logic untuk mengatasi rate limiting dan error lainnya
    max_retries = 5
    retry_count = 0
    success = False

    while not success and retry_count < max_retries:
        try:
            print(f"Menganalisis ulasan: {review_text[:50]}... (Percobaan ke-{retry_count + 1})")

            # Memanggil fungsi analisis kita
            result = analyze_review_with_granite(review_text)

            # Jika berhasil, tambahkan ulasan asli dan tandai sukses
            result['original_review'] = review_text
            analysis_results.append(result)
            success = True

        except Exception as e:
            if "status: 429" in str(e):
                try:
                    # Mencoba mengekstrak waktu tunggu dari pesan error
                    wait_time = int(''.join(filter(str.isdigit, str(e))))
                    print(f"Rate limited. Menunggu {wait_time + 2} detik dan mencoba lagi...")
                    time.sleep(wait_time + 2) # Tambahkan 2 detik untuk amannya
                except (ValueError, IndexError):
                    print("Rate limited. Tidak dapat mengekstrak waktu tunggu. Menunggu 15 detik...")
                    time.sleep(15)
                retry_count += 1
            else:
                print(f"Terjadi kesalahan: {e}. Mencatat error dan melanjutkan...")
                analysis_results.append({"error": str(e), "original_review": review_text})
                success = True

    # Jeda yang lebih panjang dan konsisten antar permintaan untuk mencegah throttling
    if success:
        print("Analisis berhasil. Menunggu 10 detik sebelum permintaan berikutnya...")
        time.sleep(10)


# ====================================================================
# Tahap 5: Pembuatan dan Analisis DataFrame Hasil
# ====================================================================

# Membuat DataFrame dari list hasil analisis
results_df = pd.DataFrame(analysis_results)

# Menampilkan 5 baris pertama dari DataFrame baru
print("\nProses analisis batch selesai!")
print("\nDataFrame Hasil Analisis AI:")
display(results_df.head())

# Menampilkan informasi tentang DataFrame baru
print("\nInformasi DataFrame Hasil:")
display(results_df.info())



In [ ]:
# ====================================================================
# Tahap 6: Pasca-Pemrosesan Hasil Analisis
# ====================================================================

# Menemukan ulasan yang gagal dianalisis
failed_reviews = results_df[results_df['error'] == 'Invalid JSON output']

# Loop untuk memperbaiki ulasan yang gagal
for index, row in failed_reviews.iterrows():
    raw_output = row['raw_output']
    original_review = row['original_review']

    try:
        # Mencoba memperbaiki output mentah menjadi JSON yang valid
        # Di sini, kita asumsikan output mentah adalah JSON yang rusak
        # Kami akan mencoba memuatnya sebagai JSON dan menanganinya
        fixed_json = json.loads(raw_output)

        # Mengisi kembali kolom di DataFrame dengan data yang diperbaiki
        results_df.loc[index, 'sentimen_keseluruhan'] = fixed_json.get('sentimen_keseluruhan')
        results_df.loc[index, 'area_fokus'] = fixed_json.get('area_fokus')
        results_df.loc[index, 'ringkasan'] = fixed_json.get('ringkasan')
        results_df.loc[index, 'error'] = None # Hapus error setelah diperbaiki

    except json.JSONDecodeError:
        # Jika perbaikan JSON gagal, kita bisa mencoba pendekatan lain
        print(f"Gagal memperbaiki ulasan: {original_review[:50]}...")
        # Kita bisa membiarkan ulasan ini sebagai "error" atau mencoba prompt ulang
        # Untuk saat ini, kita biarkan saja agar tidak memakan waktu lama

# Menampilkan informasi DataFrame setelah perbaikan
print("\nDataFrame setelah perbaikan:")
print(results_df.info())

In [ ]:
# ====================================================================
# Tahap 7: Pembersihan Akhir Data
# ====================================================================

# Menghapus kolom 'error' dan 'raw_output' karena tidak diperlukan untuk analisis lebih lanjut
results_df_clean = results_df.drop(columns=['error', 'raw_output'])

# Menghapus baris yang memiliki nilai NaN di kolom sentimen
# Ini akan membuat DataFrame kita bersih dan siap untuk analisis
results_df_clean.dropna(subset=['sentimen_keseluruhan'], inplace=True)

print("DataFrame final setelah pembersihan:")
display(results_df_clean.head())
display(results_df_clean.info())


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Set the style for the plots
sns.set_style('whitegrid')

# Create a bar plot for the overall sentiment
plt.figure(figsize=(8, 6))
sentiment_counts = results_df_clean['sentimen_keseluruhan'].value_counts()
sns.barplot(x=sentiment_counts.index, y=sentiment_counts.values, palette='viridis')
plt.title('Distribusi Sentimen Keseluruhan Ulasan (Sampel 88)', fontsize=16)
plt.xlabel('Sentimen', fontsize=12)
plt.ylabel('Jumlah Ulasan', fontsize=12)
plt.show()

In [ ]:
from collections import Counter
import ast

# Extract all focus areas from the 'area_fokus' column
all_topics = []
for topics_list in results_df_clean['area_fokus']:
    # Use ast.literal_eval to safely convert the string representation of a list into a list
    if isinstance(topics_list, str):
        topics_list = ast.literal_eval(topics_list)

    for topic_dict in topics_list:
        all_topics.append(topic_dict['topik'])

# Count the occurrences of each topic
topic_counts = Counter(all_topics)
most_common_topics = topic_counts.most_common(10) # Get the top 10 topics

# Create a DataFrame for easy plotting
topics_df = pd.DataFrame(most_common_topics, columns=['Topik', 'Jumlah'])

# Create a bar plot for the most common topics
plt.figure(figsize=(10, 8))
sns.barplot(x='Jumlah', y='Topik', data=topics_df, palette='magma')
plt.title('10 Topik yang Paling Sering Dibahas', fontsize=16)
plt.xlabel('Jumlah Pembahasan', fontsize=12)
plt.ylabel('Topik', fontsize=12)
plt.show()

In [ ]:
# Create a new DataFrame to hold sentiment counts for each topic
topic_sentiment_df = []
for topics_list in results_df_clean['area_fokus']:
    if isinstance(topics_list, str):
        topics_list = ast.literal_eval(topics_list)

    for topic_dict in topics_list:
        topic_sentiment_df.append({
            'topik': topic_dict['topik'],
            'sentimen': topic_dict['sentimen']
        })

topic_sentiment_df = pd.DataFrame(topic_sentiment_df)

# Filter for the top 5 most common topics for clarity
top_5_topics = [topic for topic, count in topic_counts.most_common(5)]
filtered_topic_df = topic_sentiment_df[topic_sentiment_df['topik'].isin(top_5_topics)]

# Create a grouped bar chart
plt.figure(figsize=(12, 8))
sns.countplot(y='topik', hue='sentimen', data=filtered_topic_df, palette='tab10')
plt.title('Sentimen Berdasarkan Topik (Top 5)', fontsize=16)
plt.xlabel('Jumlah Ulasan', fontsize=12)
plt.ylabel('Topik', fontsize=12)
plt.legend(title='Sentimen')
plt.show()
